# Use Neon as a vector database for OpenAI embeddings

This notebook guides you through using Neon Serverless Postgres as a vector database for OpenAI embeddings.

This notebook will show how to:
1. Use embeddings created by OpenAI API.
2. Store embeddings in a Neon Postgres database.
3. Convert a raw text query to an embedding with OpenAI API.
4. Use Neon with the `pgvector` extension to perform vector similarity search.

### What is Neon Serverless Postgres?

[Neon](https://neon.tech/) is Serverless Postgres built for the cloud. Neon separates compute and storage to offer modern developer features such as autoscaling, database branching, scale-to-zero, and more. You can use Neon Postgres as a vector database by installing [pgvector](https://neon.tech/docs/extensions/pgvector) or [pg_embedding](https://neon.tech/docs/extensions/pg_embedding). These are open-source PostgreSQL extensions that enable Postgres as a vector database, allowing you to perform vector similarity search in Postgres.

- [pgvector](https://neon.tech/docs/extensions/pgvector) is an open-source extension that enables vector similarity search in Postgres. It supports `ivfflat` indexes.
- [pg_embedding](https://neon.tech/docs/extensions/pg_embedding), developed and maintained by Neon, is an open-source extension that enables graph-based vector similarity search in Postgres using the Hierarchical Navigable Small World (HNSW) algorithm.

This guide uses the `pgvector` extension.

### Deployment options

Neon is a cloud-hosted Postgres database service. Refer to the [Prerequisites](#prerequisites) section for setup instructions.

## Prerequisites

Before you begin, ensure that you have the following:

1. A Neon Postgres database. You can create a Free Tier account and set up a project with a ready-to-use `neondb` database in a few simple steps. For instructions, see [Sign up](https://neon.tech/docs/get-started-with-neon/signing-up) and [Create your first project](https://neon.tech/docs/get-started-with-neon/setting-up-a-project).
2. A connection string for your `neondb` database. You can copy it from the **Connection Details** widget on the Neon **Dashboard**. see [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).
3. The `pgvector` extension. You must install the extension in Neon by running `CREATE EXTENSION vector;`. For instructions, see [Enable the pgvector extension](https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension). 
4. Your [OpenAI API key](https://platform.openai.com/account/api-keys).
5. Python and `pip`.

### Setup requirements

This notebook requires the `openai`, `psycopg2`, `pandas`, and `wget` packages. You can install them with `pip`:


In [ ]:
! pip install openai psycopg2 pandas wget

### Prepare your OpenAI API key

An OpenAI API key is required to generate vectors for documents and queries.

If you do not have an OpenAI API key, obtain one from https://platform.openai.com/account/api-keys.

Add the OpenAI API key as an operating system environment variable. Name the variable `OPENAI_API_KEY`.

For information about configuring your OpenAI API key as an environment variable, refer to [Best Practices for API Key Safety](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

### Test your OpenAPI key

In [4]:
# Test to ensure that your OpenAI API key is set correctly as an environment variable
# Note: If you run this notebook locally, you may need to reload the terminal and the notebook for the environment variables to be available.

import os

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## Connect to your Neon database

Add the database connection details to your environment variables. You can obtain the connection details from your Neon database connection string. If you get stuck, see [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).

Alternatively, you can set `psycopg2.connect` parameters.

In [12]:
import os
import psycopg2

# You can set temporary environment variables like this:
# os.environ["PGHOST"] = "ep-snowy-unit-550577.us-east-2.aws.neon.tech"
# os.environ["PGPORT"] "5432"),
# os.environ["PGDATABASE"] "neondb"),
# os.environ["PGUSER"] "user"),
# os.environ["PGPASSWORD"] "password"),

# Alternatively, you can set "psycopg2.connect" parameters

connection = psycopg2.connect(
    host=os.environ.get("PGHOST", "ep-damp-cell-18160816.us-east-2.aws.neon.tech"),
    port=os.environ.get("PGPORT", "5432"),
    database=os.environ.get("PGDATABASE", "neondb"),
    user=os.environ.get("PGUSER", "daniel"),
    password=os.environ.get("PGPASSWORD", "lsEPxw0o2eHk")
)

# Create a new cursor object
cursor = connection.cursor()

Test the connection:

In [6]:
# Execute a simple query to test the connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


This guide uses pre-computed Wikipedia article embeddings available in the OpenAI Cookbook `examples` directory so that you do not have to compute embeddings with your own OpenAI credits. 

Import the pre-computed embeddings archive file:

In [7]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB. Importing it will take some time.
wget.download(embeddings_url)

'vector_database_wikipedia_articles_embedded.zip'

Extract the downloaded archive file:

In [9]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)


# Check that the csv file exists
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)


if os.path.exists(file_path):
    print(f"The file {file_name} exists in the data directory.")
else:
    print(f"The file {file_name} does not exist in the data directory.")

The file vector_database_wikipedia_articles_embedded.csv exists in the data directory.


## Index your data

The vector table created in your database is called **articles**. Each object has **title** and **content** vectors. 

An index is defined on both the **title** and **content** vector columns.

In [13]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()

## Load data

Load the pre-computed vector data:

In [16]:
import io

# Path to your local CSV file
csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'

# Define a generator function to process the file line by line
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for the copy_expert method
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using the copy_expert method
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()

Check the size of the imported data to ensure the data has been been loaded. There should be 25000 records.

In [17]:
# Check the size of the collection
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")

Count:25000


## Search data

After the data is stored in your Neon database, query the collection for nearest neighbors. An additional `vector_name` parameter is provided to switch from "title" to "content" based similarity search. The pre-computed embeddings were created with `text-embedding-ada-002` OpenAI model, so this model must also be used to vectorize search queries.

In [18]:
def query_neon(query, collection_name, vector_name="title_vector", top_k=20):

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

Run a similarity search using the "title" vector:

In [24]:
import openai

query_results = query_neon("Greek mythology", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Greek mythology (Score: 0.998)
2. Roman mythology (Score: 0.7)
3. Greek underworld (Score: 0.637)
4. Mythology (Score: 0.635)
5. Classical mythology (Score: 0.629)
6. Japanese mythology (Score: 0.615)
7. Norse mythology (Score: 0.569)
8. Greek language (Score: 0.566)
9. Zeus (Score: 0.535)
10. List of mythologies (Score: 0.531)
11. Jupiter (mythology) (Score: 0.53)
12. Greek (Score: 0.53)
13. Gaia (mythology) (Score: 0.526)
14. Titan (mythology) (Score: 0.522)
15. Mercury (mythology) (Score: 0.521)
16. Ancient Greece (Score: 0.52)
17. Greek alphabet (Score: 0.52)
18. Venus (mythology) (Score: 0.515)
19. Pluto (mythology) (Score: 0.515)
20. Athena (Score: 0.514)


Run a similarity search using the "content" vector:

In [22]:
# Query using the "content" vector
query_results = query_neon("Famous battles in Greek history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. 222 BC (Score: 0.489)
2. Trojan War (Score: 0.458)
3. Peloponnesian War (Score: 0.456)
4. History of the Peloponnesian War (Score: 0.449)
5. 430 BC (Score: 0.441)
6. 168 BC (Score: 0.436)
7. Ancient Greece (Score: 0.429)
8. Classical Athens (Score: 0.428)
9. 499 BC (Score: 0.427)
10. Leonidas I (Score: 0.426)
11. Battle (Score: 0.421)
12. Greek War of Independence (Score: 0.421)
13. Menelaus (Score: 0.419)
14. Thebes, Greece (Score: 0.417)
15. Patroclus (Score: 0.417)
16. 427 BC (Score: 0.416)
17. 429 BC (Score: 0.413)
18. August 2 (Score: 0.412)
19. Ionia (Score: 0.411)
20. 323 (Score: 0.409)
